In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/883465/train_dataset_train.csv
!wget https://lodmedia.hb.bizmrg.com/case_files/883465/test_dataset_test.csv

In [ ]:
!pip install catboost

In [ ]:
import time
import os
import pickle
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor,NearestCentroid
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold,StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,SimpleImputer,IterativeImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics.pairwise import paired_manhattan_distances
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer
import catboost
import ast

In [ ]:
train=pd.read_csv('/content/train_dataset_train.csv')
test=pd.read_csv('/content/test_dataset_test.csv')
filt=['nd_mean_2021-04-15', 'nd_mean_2021-04-16',
       'nd_mean_2021-04-18', 'nd_mean_2021-04-22', 'nd_mean_2021-04-26',
       'nd_mean_2021-04-28', 'nd_mean_2021-04-30', 'nd_mean_2021-05-01',
       'nd_mean_2021-05-02', 'nd_mean_2021-05-08', 'nd_mean_2021-05-15',
       'nd_mean_2021-05-16', 'nd_mean_2021-05-17', 'nd_mean_2021-05-26',
       'nd_mean_2021-05-27', 'nd_mean_2021-05-29', 'nd_mean_2021-06-07',
       'nd_mean_2021-06-18', 'nd_mean_2021-06-19', 'nd_mean_2021-06-20',
       'nd_mean_2021-06-22', 'nd_mean_2021-06-25', 'nd_mean_2021-06-27',
       'nd_mean_2021-07-07', 'nd_mean_2021-07-08', 'nd_mean_2021-07-09',
       'nd_mean_2021-07-13', 'nd_mean_2021-07-15', 'nd_mean_2021-07-17',
       'nd_mean_2021-07-20', 'nd_mean_2021-07-26', 'nd_mean_2021-07-27',
       'nd_mean_2021-07-29', 'nd_mean_2021-07-31', 'nd_mean_2021-08-01',
       'nd_mean_2021-08-07', 'nd_mean_2021-08-12',
       'nd_mean_2021-04-19', 'nd_mean_2021-05-04', 'nd_mean_2021-05-07',
       'nd_mean_2021-05-21', 'nd_mean_2021-06-06', 'nd_mean_2021-06-09',
       'nd_mean_2021-06-13', 'nd_mean_2021-06-16', 'nd_mean_2021-04-25',
       'nd_mean_2021-04-27', 'nd_mean_2021-04-29', 'nd_mean_2021-06-12',
       'nd_mean_2021-06-28', 'nd_mean_2021-08-11', 'nd_mean_2021-04-20',
       'nd_mean_2021-04-23', 'nd_mean_2021-05-09', 'nd_mean_2021-05-10',
       'nd_mean_2021-05-19', 'nd_mean_2021-05-20', 'nd_mean_2021-06-03',
       'nd_mean_2021-06-04', 'nd_mean_2021-06-05', 'nd_mean_2021-06-10',
       'nd_mean_2021-07-05', 'nd_mean_2021-08-13', 'nd_mean_2021-08-27',
       'nd_mean_2021-05-03', 'nd_mean_2021-06-02', 'nd_mean_2021-07-04',
       'nd_mean_2021-08-10', 'nd_mean_2021-08-23', 'nd_mean_2021-05-24']

In [ ]:
# добавляем геокоординаты
def geo_m (geo):
  try: out=np.mean([np.array(i).mean(axis=0) for i in ast.literal_eval(geo)['coordinates']],axis=0)
  except: out=np.array([np.nan,np.nan])
  return out.tolist()
train=pd.concat((train,pd.DataFrame(map(geo_m, train['.geo'].tolist()))),axis=1)
test=pd.concat((test,pd.DataFrame(map(geo_m, test['.geo'].tolist()))),axis=1)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(train[filt+[0,1,'area']].values,
                                             train['crop'],
                                             test_size=0.1,
                                             random_state=0,
                                             stratify=train['crop'])

In [ ]:
cls=catboost.CatBoostClassifier(iterations=1000, 
                                learning_rate=0.1,
                                loss_function='MultiClassOneVsAll',
                                auto_class_weights='Balanced', 
                                use_best_model=True,
                                eval_metric='TotalF1:average=Macro',
                                bagging_temperature=0.1)

In [ ]:
cls.fit(X_train,y_train,
        eval_set=(X_val,y_val),
        verbose=10)

In [ ]:
test['crop']=cls.predict(test[filt+[0,1,'area']].values)
submit=test[['id','crop']]
submit.to_csv('submitr31.csv',index=False)